<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Cost Of Care Analytics Demo <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>
<p style='font-size:16px;font-family:Arial;color:#00233C'> With rising healthcare expenditures, there's a growing urgency to track, analyze, and optimize cost drivers across emergency room (ER), imaging, surgical, and mental health services. Traditional cost auditing methods are retrospective, siloed, and lack the granularity to enable proactive cost containment. By leveraging advanced analytics and integrated hospital datasets, a scalable, dynamic, and contextual cost intelligence solution can be achieved. However, key challenges remain: </p> <ul style='font-size:16px;font-family:Arial;color:#00233C'> <li>Fragmented cost data across procedures, diagnoses, and provider systems.</li> <li>Lack of context linking ER visits, imaging, surgeries, and mental health costs to patient pathways.</li> <li>Limited visual insights into cost trends across time, region, and condition severity.</li> <li>Need for scalable, in-database analytics to handle massive healthcare datasets.</li> </ul> <p style='font-size:16px;font-family:Arial;color:#00233C'> <b>Proposed Solution:</b><br> AI-Driven Healthcare Cost Intelligence using Teradata Vantage<br> Ingest structured data from ER visits, imaging studies, surgical procedures, and mental health conditions. Aggregate and normalize cost trends by diagnosis, time, and geography using in-database processing in Teradata Vantage. Empower analysts with interactive dashboards (X-ray/Echo trackers, Top Cost Drivers, Health Rankings) built on ipywidgets and Matplotlib for granular exploration. Integrate hospital performance scores and county-level health rankings to support value-based decision making. </p> <p style='font-size:16px;font-family:Arial;color:#00233C'> <b>Benefits:</b> <ul style='font-size:16px;font-family:Arial;color:#00233C'> <li>End-to-end cost visibility across all major healthcare services—ER, imaging, surgery, and mental health.</li> <li>Identification of high-cost hospitals and procedures with spatial and temporal granularity.</li> <li>Scalable feature engineering and analytics via Teradata’s in-database capabilities.</li> <li>Supports hospital value-based purchasing (HBVP) and patient-centric care strategies.</li> <li>Interactive and user-friendly dashboards enable deep dives into cost outliers and overutilization patterns.</li> </ul> </p>

<h1 style="font-size: 24px; font-family: Arial; color: #F0F0F0; background-color: #00233c; padding: 10pt;">Dashboard & Insights:</h1>
<p style='font-size:16px;font-family:Arial;color:#00233C'> This section highlights an interactive overview of our Cost of Care Analytics. The dashboard visualizes key metrics related to emergency room utilization, imaging and surgical procedures, and cost drivers across conditions and hospitals. It provides a holistic understanding of how healthcare costs are distributed and where intervention opportunities lie. </p> <p style='font-size:16px;font-family:Arial;color:#00233C'> Key features of the dashboard: <ul style='font-size:16px;font-family:Arial;color:#00233C'> <li>Bar charts showing annual trends in ER visits and costs across multiple categories like Avoidable, Treatable, and Not Preventable conditions.</li> <li>Stacked visualizations depicting proportionate cost contribution by diagnosis and procedure type.</li> <li>Interactive selectors to filter top diagnoses and providers by year, enabling targeted drill-downs.</li> <li>Imaging and surgical trackers to monitor frequency and cost over time for X-rays, MRIs, CT scans, and high-cost surgeries.</li> <li>Geo-mapped insights on hospital performance scores and average procedure costs across counties.</li> </ul> </p>

<p style='font-size:16px;font-family:Arial;color:#00233C'><b>Live Dashboard Link:</b>
<a href="https://superset.env.ci.clearscape.teradata.com/superset/dashboard/p/Yv7OP59LyNp/" target="_blank">Cost Of Care Dashboard</a>
</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'><b>Dashboard Snapshot:</b></p>
<img src="images/cost_of_care.jpg" width="800"/>

<p style='font-size:16px;font-family:Arial;color:#00233C'>
The dashboard is structured across multiple tabs to allow detailed investigation:
</p>

<ul style='font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Annual-Accross:</b> Tracks ER visits and preventability over time.</li>
    <li><b>Top Diagnoses:</b> Ranks primary and secondary conditions based on cost and frequency.</li>
    <li><b>Xray / Echo Tracker:</b> Flags patients and providers with excessive imaging.</li>
    <li><b>Imaging Studies:</b> Evaluates yearly and monthly volumes and costs of major scans.</li>
    <li><b>Surgical Procedures:</b> Tracks trends and maps surgery cost by region.</li>
    <li><b>Mental Health:</b> Monitors prevalence of selected mental disorders over time.</li>
    <li><b>County Health Rankings:</b> Geospatial overlay of public health indicators across counties.</li>
</ul>

<p style='font-size:16px;font-family:Arial;color:#00233C'>
Each tab enables interactive filtering by year, condition, category, and provider to customize the view and drive targeted insights.
</p>

<footer style="padding-bottom: 35px; background: #f9f9f9; border-bottom: 3px solid #00233C">
    <div style="float: left; margin-top: 14px;">ClearScape Analytics™</div>
    <div style="float: right;">
        <div style="float: left; margin-top: 14px;">Copyright © Teradata Corporation - 2025. All Rights Reserved.</div>
    </div>
</footer>